In [4]:
import simpy
import numpy as np

import itertools

2) Discrete Event Simulation

In [17]:
# https://simpy.readthedocs.io/en/latest/examples/carwash.html
np.random.seed(3)

# λ – the arrival rate into the system as a whole.
arrival_rate = 0.5

# μ – the capacity of each of n equal servers. 
capacity = 1

# ρ represents the system load. ρ=λ/(nμ). 

# Simulation time
sim_time = 10


wait_times_n1 = []
wait_times_n2 = []
wait_times_n4 = []


class Queue: 
    def __init__(self, env, n_servers, serving_time):
        self.env = env
        self.servers = simpy.Resource(env, n_servers)
        self.serving_time = serving_time

    def serving(self, person):
        yield self.env.timeout(self.serving_time)
        print(person)

def person(env, name, queue):
    with queue.servers.request() as request:
        yield request

        yield env.process(queue.serving(name))

def setup(env, n_servers, serving_time, arrival_rate):
    waiting = Queue(env, n_servers, serving_time)

    person_count = itertools.count()

    # Create 1 initial person
    env.process(person(env, f'Person {next(person_count)}', waiting))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(arrival_rate)
        env.process(person(env, f'Person {next(person_count)}', waiting))



env = simpy.Environment()
env.process(setup(env, 1, 1, 1))

# Execute!
env.run(until=10)


Person 0
Person 1
Person 2
Person 3
Person 4
Person 5
Person 6
Person 7
Person 8
